In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime

In [4]:
files = os.listdir("../data/tramdo")

In [6]:
required_columns = ['pm25', 'pm10', 'o3', 'no2', 'so2', 'co', 'aqi']

In [10]:
def classify_aqi(aqi):
    if 0 <= aqi <= 50:
        return 'Tốt'
    elif 51 <= aqi <= 100:
        return 'Vừa Phải'
    elif 101 <= aqi <= 150:
        return 'Không tốt cho người nhạy cảm'
    elif 151 <= aqi <= 200:
        return 'Không tốt'
    elif 201 <= aqi <= 300:
        return 'Rất không tốt'
    else:
        return 'Nguy hiểm'

In [14]:
files[0]

'Ba Đình - Hà Nội.csv'

In [64]:
for file in files:
    # print(file)
    # Đọc dữ liệu từ tệp CSV
    file_path = f"../data/tramdo/{file}"
    data = pd.read_csv(file_path)

    try:
        data['date'] = pd.to_datetime(data['date'])

    except ValueError:
        objects = [datetime.strptime(date, "%d/%m/%Y") for date in data['date']]
        # Chuyển đổi sang định dạng mới "dd/mm/yyyy"
        data["date"] = [date.strftime("%d/%m/%Y") for date in objects]
        # Chuyển đổi cột 'date' sang định dạng datetime
        data['date'] = pd.to_datetime(data['date'], format="%d/%m/%Y")

    # Reset lại index
    data = data.reset_index(drop=True)

    # Sửa tên cột
    data = data.rename(columns=lambda x: x.strip())
    data[data.columns[1:]] = data[data.columns[1:]].replace(' ', 0) #ép kiểu cột sang int
    data[data.columns[1:]] = data[data.columns[1:]].astype('double')

    # Thêm cột mới nếu chưa xuất hiện
    for column in required_columns:
        if column not in data.columns:
            data[column] = 0

    # Thêm cột 'aqi' bằng cách lấy giá trị lớn nhất
    data['aqi'] = data[required_columns].max(axis=1)
    # Đảm bảo thứ tự các cột là 'pm25', 'pm10', 'o3', 'no2', 'so2', 'co', 'aqi'
    data = data[['date', 'pm25', 'pm10', 'o3', 'no2', 'so2', 'co', 'aqi']]

    # Áp dụng hàm phân loại AQI cho cột 'aqi'
    data['mức độ ô nhiễm'] = data['aqi'].apply(classify_aqi)
    
    # Thêm cột 'địa điểm trạm đo' từ tên tệp
    data.insert(0, "địa điểm trạm đo", file.split("-")[0].strip())

    # Lưu DataFrame đã cập nhật vào tệp CSV mới
    data.to_csv(file_path, index=False)
print("Okey!!!!!!!")

Ba Đình - Hà Nội.csv
Biên Hòa - Đồng Nai.csv


ValueError: time data '2023-01-01' does not match format '%d/%m/%Y'